In [76]:
import causalnex
import networkx as nx
from causalnex.structure.dynotears import from_pandas_dynamic
from causalnex.structure.pytorch.notears import from_pandas
from causalnex.plots import plot_structure, NODE_STYLE, EDGE_STYLE
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import torch
import warnings
warnings.filterwarnings('ignore')

In [77]:
df_timestamp = pd.read_csv("SWaT_Dataset/2015/Generated_csv/timestampV1.csv",index_col=0).reset_index(drop=True)
df_att_timestamp = df_timestamp[247500:274500].reset_index(drop=True)
df_att_timestamp

,Timestamp
0,2015-12-25 13:15:00
1,2015-12-25 13:15:01
2,2015-12-25 13:15:02
3,2015-12-25 13:15:03
4,2015-12-25 13:15:04
...,...
26995,2015-12-25 20:44:55
26996,2015-12-25 20:44:56
26997,2015-12-25 20:44:57
26998,2015-12-25 20:44:58


In [118]:
df_attack_train = pd.read_csv('SWaT_Dataset/2015/final_csv/attack_data_trainV5.csv',index_col=0).reset_index(drop=True)
# df_attack_train.drop(columns='Normal/Attack', inplace=True)
# df_attack_train['Timestamp'] = df_att_timestamp

df_attack_train

,Timestamp,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,...,P501,P502,PIT501,PIT502,PIT503,FIT601,P601,P602,P603,IsAttack
0,2015-12-25 13:15:00,2.427057,522.8467,2,2,1,262.0161,8.396437,328.6337,2.445391,...,2,1,250.8652,1.649953,189.5988,0.000128,1,1,1,0
1,2015-12-25 13:15:01,2.446274,522.8860,2,2,1,262.0161,8.396437,328.6337,2.445391,...,2,1,250.8652,1.649953,189.6789,0.000128,1,1,1,0
2,2015-12-25 13:15:02,2.489191,522.8467,2,2,1,262.0161,8.394514,328.6337,2.442316,...,2,1,250.8812,1.649953,189.6789,0.000128,1,1,1,0
3,2015-12-25 13:15:03,2.534350,522.9645,2,2,1,262.0161,8.394514,328.6337,2.442316,...,2,1,250.8812,1.649953,189.6148,0.000128,1,1,1,0
4,2015-12-25 13:15:04,2.569260,523.4748,2,2,1,262.0161,8.394514,328.6337,2.443085,...,2,1,250.8812,1.649953,189.5027,0.000128,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33249,2015-12-25 22:29:09,0.000000,714.9497,1,2,1,189.7270,8.497373,351.4483,2.462051,...,2,1,249.5834,1.025214,187.9807,0.000000,1,1,1,0
33250,2015-12-25 22:29:10,0.000000,714.2824,1,2,1,189.7270,8.497373,351.4483,2.462051,...,2,1,249.5834,1.025214,187.9807,0.000000,1,1,1,0
33251,2015-12-25 22:29:11,0.000000,713.6151,1,2,1,189.7270,8.497373,351.4483,2.462692,...,2,1,249.5834,0.881043,187.9807,0.000000,1,1,1,0
33252,2015-12-25 22:29:12,0.000000,712.8301,1,2,1,189.7270,8.497373,351.4483,2.464742,...,2,1,249.5834,0.881043,187.9807,0.000000,1,1,1,0


In [119]:
df_normal_train = pd.read_csv('SWaT_Dataset/2015/final_csv/normal_data_trainV5.csv',index_col=0).reset_index(drop=True)
df_normal_train

,Timestamp,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,...,P501,P502,PIT501,PIT502,PIT503,FIT601,P601,P602,P603,IsAttack
0,2015-12-22 16:30:00,0.000000,124.3135,1,1,1,251.9226,8.313446,312.7916,0.000000,...,1,1,9.100231,0.000000,3.3485,0.000256,1,1,1,0
1,2015-12-22 16:30:01,0.000000,124.3920,1,1,1,251.9226,8.313446,312.7916,0.000000,...,1,1,9.100231,0.000000,3.3485,0.000256,1,1,1,0
2,2015-12-22 16:30:02,0.000000,124.4705,1,1,1,251.9226,8.313446,312.7916,0.000000,...,1,1,9.100231,0.000000,3.3485,0.000256,1,1,1,0
3,2015-12-22 16:30:03,0.000000,124.6668,1,1,1,251.9226,8.313446,312.7916,0.000000,...,1,1,9.100231,0.000000,3.3485,0.000256,1,1,1,0
4,2015-12-22 16:30:04,0.000000,124.5098,1,1,1,251.9226,8.313446,312.7916,0.000000,...,1,1,9.100231,0.000000,3.3485,0.000256,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247495,2015-12-25 13:14:55,2.534029,506.0858,2,2,1,266.3419,8.353820,327.7108,2.448467,...,2,1,250.865200,1.137347,189.9673,0.000000,1,1,1,0
247496,2015-12-25 13:14:56,2.522499,505.8896,2,2,1,266.3419,8.353820,327.7108,2.448467,...,2,1,250.865200,1.137347,190.0795,0.000000,1,1,1,0
247497,2015-12-25 13:14:57,2.510329,505.6933,2,2,1,266.3419,8.353820,327.7878,2.448467,...,2,1,250.865200,1.137347,190.0795,0.000000,1,1,1,0
247498,2015-12-25 13:14:58,2.494635,505.6148,2,2,1,266.3419,8.353820,327.7878,2.448595,...,2,1,250.865200,1.137347,190.1596,0.000000,1,1,1,0


In [120]:
df_test = pd.read_csv('SWaT_Dataset/2015/final_csv/test_data_v5.csv',index_col=0).reset_index(drop=True)
df_test

,Timestamp,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,...,P501,P502,PIT501,PIT502,PIT503,FIT601,P601,P602,P603,IsAttack
0,2015-12-25 22:29:14,2.469013,505.8110,2,2,1,266.3419,8.353820,327.7878,2.448595,...,2,1,250.8652,1.201422,190.1596,0.0,1,1,1,0
1,2015-12-25 22:29:15,2.456202,506.4783,2,2,1,266.3419,8.353820,327.7878,2.446929,...,2,1,250.8652,1.201422,190.1596,0.0,1,1,1,0
2,2015-12-25 22:29:16,2.438587,506.6353,2,2,1,266.3419,8.353820,327.7878,2.446929,...,2,1,250.8652,1.201422,190.0154,0.0,1,1,1,0
3,2015-12-25 22:29:17,2.423854,507.3419,2,2,1,266.3419,8.353820,327.7878,2.446929,...,2,1,250.7210,1.201422,189.9353,0.0,1,1,1,0
4,2015-12-25 22:29:18,2.413926,507.7737,2,2,1,266.3419,8.353820,327.7878,2.446929,...,2,1,250.6409,1.201422,189.8552,0.0,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664160,2016-01-02 07:19:34,2.559972,519.5495,2,2,1,168.0979,8.638683,301.9226,2.459488,...,2,1,251.1535,0.865024,189.0220,0.0,1,1,1,0
664161,2016-01-02 07:19:35,2.549082,520.4131,2,2,1,168.0979,8.638683,301.9226,2.459488,...,2,1,251.0734,0.865024,188.9259,0.0,1,1,1,0
664162,2016-01-02 07:19:36,2.531467,520.6878,2,2,1,168.0979,8.638683,301.9226,2.460129,...,2,1,251.0734,0.865024,188.9259,0.0,1,1,1,0
664163,2016-01-02 07:19:37,2.521218,520.7271,2,2,1,168.0979,8.638683,301.9226,2.460129,...,2,1,251.0734,0.865024,188.9259,0.0,1,1,1,0


In [121]:
df_attack_train['Timestamp'] = pd.to_datetime(df_attack_train['Timestamp'])
df_normal_train['Timestamp'] = pd.to_datetime(df_normal_train['Timestamp'])
df_test['Timestamp'] = pd.to_datetime(df_test['Timestamp'])

In [15]:
df_attack_train['Timestamp'] = df_attack_train['Timestamp'].apply(lambda x:x.value)
df_attack_train.drop(columns='IsAttack', inplace=True)


In [16]:
struct_model_attack = from_pandas_dynamic(df_attack_train, 4)

In [17]:
df_normal_train['Timestamp'] = df_normal_train['Timestamp'].apply(lambda x:x.value)
df_normal_train.drop(columns='IsAttack', inplace=True)


In [18]:
struct_model_normal = from_pandas_dynamic(df_normal_train, 4)

In [122]:
y_true = df_test['IsAttack']

In [123]:
df_test.drop(columns=['IsAttack'], inplace=True)

In [21]:
def structural_hamming_distance(dag1, dag2):
    # Convert DAGs to sets of edges
    edges1 = set(dag1.edges)
    edges2 = set(dag2.edges)
    
    # Compute the difference in edges (symmetric difference)
    symmetric_diff = edges1 ^ edges2
    
    # Return the size of the symmetric difference
    return len(symmetric_diff)

In [124]:
y_true_segs = []
for i in range(0, len(y_true), 900):
    # print(sum(y_true[i:i+900]))
    if (sum(y_true[i:i+900])>0):
        y_true_segs.append(1)
    else:
        y_true_segs.append(0)

In [85]:
sum(y_true_segs)

43

In [126]:
y_pred = []
for i in range(0, len(df_test),900):
    df_struct_test = df_test[i:i+900]
    df_struct_test['Timestamp'] = df_struct_test['Timestamp'].apply(lambda x: x.value)
    struct_model_test = from_pandas_dynamic(df_struct_test, 4)

    shd_TA = structural_hamming_distance(struct_model_attack, struct_model_test)
    shd_TN = structural_hamming_distance(struct_model_normal, struct_model_test)

    if shd_TA<shd_TN:
        y_pred.append(1)
    else:
        y_pred.append(0)

In [127]:
from sklearn.metrics import precision_score, f1_score, recall_score, accuracy_score, roc_auc_score

precision = precision_score(y_true_segs, y_pred)
recall = recall_score(y_true_segs, y_pred)
roc = roc_auc_score(y_true_segs, y_pred)
acc = accuracy_score(y_true_segs, y_pred)
f1 = f1_score(y_true_segs, y_pred)

print(f"Precision : {precision}")
print(f"Recall : {recall}")
print(f" ROC : {roc}")
print(f"Accuracy : {acc}")
print(f"F1_Score : {f1}")

Precision : 1.0
Recall : 0.627906976744186
 ROC : 0.813953488372093
Accuracy : 0.978319783197832
F1_Score : 0.7714285714285715


In [159]:
df_test2 = pd.read_csv('../../SWaT_Dataset/2015/Generated_csv/test_data_v3.csv',index_col=0).reset_index(drop=True)
df_test2

,Timestamp,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,...,P501,P502,PIT501,PIT502,PIT503,FIT601,P601,P602,P603,IsAttack
0,2015-12-25 20:45:00,0.000000,633.5002,1,2,1,174.1861,8.620419,302.2302,2.453337,...,2,1,249.2630,0.816967,187.5961,0.000000,1,1,1,0
1,2015-12-25 20:45:01,2.589757,526.2617,2,2,1,231.3189,8.482954,326.3522,2.451799,...,2,1,250.7850,0.784929,189.0541,0.000000,1,1,1,0
2,2015-12-25 20:45:02,2.451078,511.9345,2,2,1,261.8559,8.387465,329.2745,2.439625,...,2,1,251.0414,1.730048,189.8071,0.000128,1,1,1,0
3,2015-12-25 20:45:03,2.430260,506.0073,2,2,1,258.2992,8.375288,332.6070,2.441803,...,2,1,247.4366,0.961138,186.4426,0.000064,1,1,1,0
4,2015-12-25 20:45:04,0.000000,680.1326,1,2,1,259.9333,8.369841,332.6070,2.451671,...,2,1,247.2763,1.601897,186.4586,0.000128,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
670393,2016-01-02 14:59:34,2.668866,509.8933,2,2,1,271.0844,8.350295,329.9667,2.448595,...,2,1,253.3165,1.105309,192.0822,0.000064,1,1,1,0
670394,2016-01-02 14:59:35,2.571182,750.1987,2,1,1,183.3825,8.672008,302.7429,0.000000,...,2,1,250.3044,1.009195,188.4132,0.000000,1,1,1,0
670395,2016-01-02 14:59:36,2.564135,501.3755,2,2,1,264.4514,8.358306,331.6073,2.423606,...,2,1,247.8531,1.089290,187.1155,0.000000,1,1,1,0
670396,2016-01-02 14:59:37,2.555168,671.2614,2,1,1,176.1087,8.663676,300.0513,0.000000,...,2,1,252.2751,0.800948,190.0955,0.000000,1,1,1,0


In [348]:

df_time = df_test2['Timestamp']

group_size = 1400

# Calculate the number of groups
num_groups = len(df_test2) // group_size

# Create an array to store the shuffled indices
shuffled_indices = np.arange(len(df_test2))

# Shuffle the indices within each group
for i in range(num_groups):
    start_index = i * group_size
    end_index = (i + 1) * group_size
    np.random.shuffle(shuffled_indices[start_index:end_index])

# Use the shuffled indices to reorder the DataFrame
shuffled_df = df_test2.iloc[shuffled_indices].reset_index(drop=True)
shuffled_df['Timestamp'] = df_time
shuffled_df

,Timestamp,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,...,P501,P502,PIT501,PIT502,PIT503,FIT601,P601,P602,P603,IsAttack
0,2015-12-25 20:45:00,2.446594,726.3723,2,1,1,266.0536,8.422392,316.5855,0.000000,...,2,1,246.0587,1.169384,185.6575,0.000064,1,1,1,0
1,2015-12-25 20:45:01,0.000000,693.0074,1,2,1,270.6998,8.361190,333.1197,2.458976,...,2,1,253.3805,1.185403,192.0661,0.000064,1,1,1,0
2,2015-12-25 20:45:02,2.531467,698.0710,2,1,1,263.2979,8.417906,318.7644,0.000000,...,2,1,249.6956,1.137347,188.7817,0.000064,1,1,1,0
3,2015-12-25 20:45:03,2.582711,513.4261,2,2,1,260.3820,8.377211,331.0433,2.427578,...,2,1,250.1602,1.169384,188.7657,0.000064,1,1,1,0
4,2015-12-25 20:45:04,0.000000,795.4179,1,2,1,258.4914,8.382338,334.7859,2.463204,...,2,1,248.1415,1.041233,187.0354,0.000064,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
670393,2016-01-02 14:59:34,2.668866,509.8933,2,2,1,271.0844,8.350295,329.9667,2.448595,...,2,1,253.3165,1.105309,192.0822,0.000064,1,1,1,0
670394,2016-01-02 14:59:35,2.571182,750.1987,2,1,1,183.3825,8.672008,302.7429,0.000000,...,2,1,250.3044,1.009195,188.4132,0.000000,1,1,1,0
670395,2016-01-02 14:59:36,2.564135,501.3755,2,2,1,264.4514,8.358306,331.6073,2.423606,...,2,1,247.8531,1.089290,187.1155,0.000000,1,1,1,0
670396,2016-01-02 14:59:37,2.555168,671.2614,2,1,1,176.1087,8.663676,300.0513,0.000000,...,2,1,252.2751,0.800948,190.0955,0.000000,1,1,1,0


In [253]:
sum(shuffled_df[10800:11700]['IsAttack'])

307

In [349]:
y_true = shuffled_df['IsAttack']
shuffled_df.drop(columns=['IsAttack'], inplace=True)

In [350]:
sum(y_true)

36600

In [354]:
y_true_segs = []
for i in range(0, len(y_true), 900):
    if sum(y_true[i:i+900])>225:
        y_true_segs.append(1)
    else:
        y_true_segs.append(0)

In [352]:
y_pred = []
for i in range(0, len(df_test),900):
    df_struct_test = df_test[i:i+900]
    df_struct_test['Timestamp'] = df_struct_test['Timestamp'].apply(lambda x: x.value)
    struct_model_test = from_pandas_dynamic(df_struct_test, 4)

    shd_TA = structural_hamming_distance(struct_model_attack, struct_model_test)
    shd_TN = structural_hamming_distance(struct_model_normal, struct_model_test)

    if shd_TA<shd_TN:
        y_pred.append(1)
    else:
        y_pred.append(0)


In [356]:
precision = precision_score(y_true_segs, y_pred)
recall = recall_score(y_true_segs, y_pred)
roc = roc_auc_score(y_true_segs, y_pred)
acc = accuracy_score(y_true_segs, y_pred)
f1 = f1_score(y_true_segs, y_pred)

print(f"Precision : {precision}")
print(f"Recall : {recall}")
print(f" ROC : {roc}")
print(f"Accuracy : {acc}")
print(f"F1_Score : {f1}")

Precision : 0.8360655737704918
Recall : 0.6219512195121951
 ROC : 0.8034341316263842
Accuracy : 0.9449664429530201
F1_Score : 0.7132867132867133


In [21]:
sum(y_true_segs), sum(y_pred)

(116, 61)

In [106]:
#Baseline_Models
df_attack_train_baseline = pd.read_csv('SWaT_Dataset/2015/final_csv/attack_data_trainV5.csv',index_col=0).reset_index(drop=True)
df_normal_train_baseline = pd.read_csv('SWaT_Dataset/2015/final_csv/normal_data_trainV5.csv',index_col=0).reset_index(drop=True)
df_test_baseline = pd.read_csv('SWaT_Dataset/2015/final_csv/test_data_v5.csv',index_col=0).reset_index(drop=True)
# df_normal_train_baseline['IsAttack'] = 0



In [107]:
df_test_baseline

,Timestamp,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,...,P501,P502,PIT501,PIT502,PIT503,FIT601,P601,P602,P603,IsAttack
0,2015-12-25 22:29:14,2.469013,505.8110,2,2,1,266.3419,8.353820,327.7878,2.448595,...,2,1,250.8652,1.201422,190.1596,0.0,1,1,1,0
1,2015-12-25 22:29:15,2.456202,506.4783,2,2,1,266.3419,8.353820,327.7878,2.446929,...,2,1,250.8652,1.201422,190.1596,0.0,1,1,1,0
2,2015-12-25 22:29:16,2.438587,506.6353,2,2,1,266.3419,8.353820,327.7878,2.446929,...,2,1,250.8652,1.201422,190.0154,0.0,1,1,1,0
3,2015-12-25 22:29:17,2.423854,507.3419,2,2,1,266.3419,8.353820,327.7878,2.446929,...,2,1,250.7210,1.201422,189.9353,0.0,1,1,1,0
4,2015-12-25 22:29:18,2.413926,507.7737,2,2,1,266.3419,8.353820,327.7878,2.446929,...,2,1,250.6409,1.201422,189.8552,0.0,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664160,2016-01-02 07:19:34,2.559972,519.5495,2,2,1,168.0979,8.638683,301.9226,2.459488,...,2,1,251.1535,0.865024,189.0220,0.0,1,1,1,0
664161,2016-01-02 07:19:35,2.549082,520.4131,2,2,1,168.0979,8.638683,301.9226,2.459488,...,2,1,251.0734,0.865024,188.9259,0.0,1,1,1,0
664162,2016-01-02 07:19:36,2.531467,520.6878,2,2,1,168.0979,8.638683,301.9226,2.460129,...,2,1,251.0734,0.865024,188.9259,0.0,1,1,1,0
664163,2016-01-02 07:19:37,2.521218,520.7271,2,2,1,168.0979,8.638683,301.9226,2.460129,...,2,1,251.0734,0.865024,188.9259,0.0,1,1,1,0


In [108]:
df_baseline_timestamp = pd.concat([df_normal_train_baseline['Timestamp'], df_attack_train_baseline['Timestamp'], df_test['Timestamp']]).reset_index(drop=True)
df_baseline_timestamp

0         2015-12-22 16:30:00
1         2015-12-22 16:30:01
2         2015-12-22 16:30:02
3         2015-12-22 16:30:03
4         2015-12-22 16:30:04
                 ...         
944914    2016-01-02 07:19:34
944915    2016-01-02 07:19:35
944916    2016-01-02 07:19:36
944917    2016-01-02 07:19:37
944918    2016-01-02 07:19:38
Name: Timestamp, Length: 944919, dtype: object

In [109]:
df_baseline_train = pd.concat([df_normal_train_baseline, df_attack_train_baseline, df_test_baseline[:270602]]).reset_index(drop=True)
df_baseline_test = df_test_baseline[270602:].reset_index(drop=True)

In [110]:
df_baseline_train

,Timestamp,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,...,P501,P502,PIT501,PIT502,PIT503,FIT601,P601,P602,P603,IsAttack
0,2015-12-22 16:30:00,0.0,124.3135,1,1,1,251.9226,8.313446,312.7916,0.000000,...,1,1,9.100231,0.000000,3.3485,0.000256,1,1,1,0
1,2015-12-22 16:30:01,0.0,124.3920,1,1,1,251.9226,8.313446,312.7916,0.000000,...,1,1,9.100231,0.000000,3.3485,0.000256,1,1,1,0
2,2015-12-22 16:30:02,0.0,124.4705,1,1,1,251.9226,8.313446,312.7916,0.000000,...,1,1,9.100231,0.000000,3.3485,0.000256,1,1,1,0
3,2015-12-22 16:30:03,0.0,124.6668,1,1,1,251.9226,8.313446,312.7916,0.000000,...,1,1,9.100231,0.000000,3.3485,0.000256,1,1,1,0
4,2015-12-22 16:30:04,0.0,124.5098,1,1,1,251.9226,8.313446,312.7916,0.000000,...,1,1,9.100231,0.000000,3.3485,0.000256,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
551351,2015-12-29 00:49:11,0.0,655.9136,1,2,1,261.5996,8.353179,333.0428,2.449364,...,2,1,246.667500,1.601897,185.9299,0.000128,1,1,1,0
551352,2015-12-29 00:49:12,0.0,655.2855,1,2,1,261.5996,8.353179,333.0428,2.449364,...,2,1,246.667500,1.601897,185.9299,0.000128,1,1,1,0
551353,2015-12-29 00:49:13,0.0,654.8538,1,2,1,261.5996,8.353179,333.0428,2.449364,...,2,1,246.619500,1.601897,185.9299,0.000128,1,1,1,0
551354,2015-12-29 00:49:14,0.0,654.3042,1,2,1,261.5996,8.353179,333.0428,2.451286,...,2,1,246.619500,1.457726,185.9299,0.000128,1,1,1,0


In [111]:
group_size = 1600

# Calculate the number of groups
num_groups = len(df_baseline_train) // group_size

# Create an array to store the shuffled indices
shuffled_indices = np.arange(len(df_baseline_train))

# Shuffle the indices within each group
for i in range(num_groups):
    start_index = i * group_size
    end_index = (i + 1) * group_size
    np.random.shuffle(shuffled_indices[start_index:end_index])

# Use the shuffled indices to reorder the DataFrame
shuffled_df = df_baseline_train.iloc[shuffled_indices].reset_index(drop=True)
shuffled_df['Timestamp'] = df_baseline_timestamp[:551356]
df_baseline_train = shuffled_df

In [112]:
y_train = df_baseline_train['IsAttack']
df_baseline_train.drop(columns='IsAttack', inplace=True)
X_train = df_baseline_train
X_train

,Timestamp,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,...,FIT504,P501,P502,PIT501,PIT502,PIT503,FIT601,P601,P602,P603
0,2015-12-22 16:30:00,2.638119,264.9169,2,2,1,264.5796,8.262177,483.7221,2.467946,...,0.000000,2,1,195.927300,3.492135,161.368900,0.000128,1,1,1
1,2015-12-22 16:30:01,0.000000,124.3920,1,1,1,251.7303,8.311523,313.7657,0.000000,...,0.000000,1,1,9.068188,0.000000,3.268393,0.000256,1,1,1
2,2015-12-22 16:30:02,0.000000,123.9602,1,1,1,251.9226,8.308640,313.3299,0.000000,...,0.000000,1,1,9.068188,0.000000,3.348500,0.000256,1,1,1
3,2015-12-22 16:30:03,2.440188,217.4995,2,1,1,252.5634,8.309601,316.8675,0.000000,...,0.000000,1,1,10.430020,0.000000,4.614202,0.000128,1,1,1
4,2015-12-22 16:30:04,2.425456,204.8994,2,1,1,252.5634,8.312805,316.9956,0.000000,...,0.000000,1,1,10.430020,0.000000,4.614202,0.000128,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
551351,2015-12-29 00:49:11,0.000000,655.9136,1,2,1,261.5996,8.353179,333.0428,2.449364,...,0.306633,2,1,246.667500,1.601897,185.929900,0.000128,1,1,1
551352,2015-12-29 00:49:12,0.000000,655.2855,1,2,1,261.5996,8.353179,333.0428,2.449364,...,0.306633,2,1,246.667500,1.601897,185.929900,0.000128,1,1,1
551353,2015-12-29 00:49:13,0.000000,654.8538,1,2,1,261.5996,8.353179,333.0428,2.449364,...,0.306633,2,1,246.619500,1.601897,185.929900,0.000128,1,1,1
551354,2015-12-29 00:49:14,0.000000,654.3042,1,2,1,261.5996,8.353179,333.0428,2.451286,...,0.306633,2,1,246.619500,1.457726,185.929900,0.000128,1,1,1


In [113]:
y_test = df_baseline_test['IsAttack']
df_baseline_test.drop(columns='IsAttack', inplace=True)
X_test = df_baseline_test
X_test

,Timestamp,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,...,FIT504,P501,P502,PIT501,PIT502,PIT503,FIT601,P601,P602,P603
0,2015-12-29 00:49:16,0.000000,653.1266,1,2,1,261.5996,8.355102,333.0428,2.452055,...,0.305352,2,1,246.6195,1.505783,185.9299,0.000128,1,1,1
1,2015-12-29 00:49:17,0.000000,652.5771,1,2,1,261.5996,8.355102,333.0428,2.452055,...,0.304647,2,1,246.6195,1.601897,185.9299,0.000128,1,1,1
2,2015-12-29 00:49:18,0.000000,652.1846,1,2,1,261.5996,8.355102,333.0428,2.452055,...,0.304647,2,1,246.6195,1.601897,185.9299,0.000128,1,1,1
3,2015-12-29 00:49:19,0.000000,652.1061,1,2,1,261.5996,8.355102,333.0428,2.452055,...,0.306441,2,1,246.6195,1.601897,185.9299,0.000128,1,1,1
4,2015-12-29 00:49:20,0.000000,651.9098,1,2,1,261.5996,8.355102,333.0428,2.452055,...,0.306569,2,1,246.6195,1.601897,185.9299,0.000128,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393558,2016-01-02 07:19:34,2.559972,519.5495,2,2,1,168.0979,8.638683,301.9226,2.459488,...,0.306569,2,1,251.1535,0.865024,189.0220,0.000000,1,1,1
393559,2016-01-02 07:19:35,2.549082,520.4131,2,2,1,168.0979,8.638683,301.9226,2.459488,...,0.306569,2,1,251.0734,0.865024,188.9259,0.000000,1,1,1
393560,2016-01-02 07:19:36,2.531467,520.6878,2,2,1,168.0979,8.638683,301.9226,2.460129,...,0.306569,2,1,251.0734,0.865024,188.9259,0.000000,1,1,1
393561,2016-01-02 07:19:37,2.521218,520.7271,2,2,1,168.0979,8.638683,301.9226,2.460129,...,0.307978,2,1,251.0734,0.865024,188.9259,0.000000,1,1,1


In [114]:
X_train['Timestamp'] = pd.to_datetime(X_train['Timestamp'])
X_test['Timestamp'] = pd.to_datetime(X_test['Timestamp'])
X_train['Timestamp'] = X_train['Timestamp'].apply(lambda x:x.value)
X_test['Timestamp'] = X_test['Timestamp'].apply(lambda x:x.value)

In [116]:
from pyod.models.ecod import ECOD
from pyod.utils.data import generate_data
from pyod.utils.data import evaluate_print

ecod_model = ECOD()

ecod_model.fit(X_train)

y_train_preds = ecod_model.labels_

y_train_Scores = ecod_model.decision_scores_

y_test_preds = ecod_model.predict(X_test)

y_test_Scores = ecod_model.decision_function(X_test)

y_true = []
for i in range(0, len(y_test),900):
    if (sum(y_test[i:i+900])>0):
        y_true.append(1)
    else:
        y_true.append(0)

y_pred = []
for i in range(0, len(y_test),900):
    if (sum(y_test_preds[i:i+900])>0):
        y_pred.append(1)
    else:
        y_pred.append(0)

precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1Score = f1_score(y_true, y_pred)
roc = roc_auc_score(y_true, y_pred)
acc = accuracy_score(y_true, y_pred)

print(f"Precision : {precision}")
print(f"Recall : {recall}")
print(f" ROC : {roc}")
print(f"Accuracy : {acc}")
print(f"F1_Score : {f1Score}")

Precision : 0.1483050847457627
Recall : 0.8974358974358975
 ROC : 0.6968382494698284
Accuracy : 0.5319634703196348
F1_Score : 0.2545454545454545


In [289]:
df_timestamp = pd.read_csv('../../SWaT_Dataset/2015/Generated_csv/timestampV1.csv', index_col=0).reset_index(drop=True)
df_timestamp

,Timestamp
0,2015-12-22 16:30:00
1,2015-12-22 16:30:01
2,2015-12-22 16:30:02
3,2015-12-22 16:30:03
4,2015-12-22 16:30:04
...,...
944914,2016-01-02 14:59:55
944915,2016-01-02 14:59:56
944916,2016-01-02 14:59:57
944917,2016-01-02 14:59:58


In [290]:
df_baseline['Timestamp'] = df_timestamp[:545102]
df_baseline

,Timestamp,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,...,P501,P502,PIT501,PIT502,PIT503,FIT601,P601,P602,P603,IsAttack
0,2015-12-22 16:30:00,2.423854,502.5531,2,2,1,266.5342,8.379133,329.7872,2.439881,...,2,1,251.7784,1.105309,190.3839,0.000064,1,1,1,0
1,2015-12-22 16:30:01,0.000000,719.4246,1,2,1,188.6055,8.520123,375.3140,2.473200,...,2,1,250.5608,1.025214,188.5734,0.000064,1,1,1,0
2,2015-12-22 16:30:02,2.447875,575.8380,2,1,1,199.6283,8.575878,314.2784,0.000000,...,2,1,247.4366,1.025214,186.3625,0.000000,1,1,1,0
3,2015-12-22 16:30:03,2.428658,512.0915,2,2,1,178.8644,8.590618,315.7652,2.449108,...,2,1,250.6729,0.945119,188.9259,0.000000,1,1,1,0
4,2015-12-22 16:30:04,2.545239,505.3008,2,2,1,264.4194,8.348372,331.4535,2.429501,...,2,1,246.5714,1.105309,185.8177,0.000064,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545097,2015-12-28 23:54:57,0.000000,813.0031,1,1,1,266.6624,8.446745,319.3284,0.000000,...,2,1,251.1856,1.057252,190.0154,0.000064,1,1,1,0
545098,2015-12-28 23:54:58,0.000000,681.7811,1,2,1,229.7167,8.466612,332.6583,2.467305,...,2,1,250.6248,0.688816,189.0220,0.000000,1,1,1,0
545099,2015-12-28 23:54:59,2.515453,518.5289,2,2,1,261.9200,8.378813,329.7616,2.427450,...,2,1,249.8398,1.185403,188.7337,0.000000,1,1,1,0
545100,2015-12-28 23:55:00,2.523460,518.9214,2,2,1,177.4225,8.611446,303.7682,2.445904,...,2,1,250.9773,0.816967,188.8458,0.000000,1,1,1,0


In [291]:
y_train = df_baseline['IsAttack']
df_baseline.drop(columns='IsAttack', inplace=True)
X_train = df_baseline
X_train

,Timestamp,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,...,FIT504,P501,P502,PIT501,PIT502,PIT503,FIT601,P601,P602,P603
0,2015-12-22 16:30:00,2.423854,502.5531,2,2,1,266.5342,8.379133,329.7872,2.439881,...,0.308491,2,1,251.7784,1.105309,190.3839,0.000064,1,1,1
1,2015-12-22 16:30:01,0.000000,719.4246,1,2,1,188.6055,8.520123,375.3140,2.473200,...,0.307978,2,1,250.5608,1.025214,188.5734,0.000064,1,1,1
2,2015-12-22 16:30:02,2.447875,575.8380,2,1,1,199.6283,8.575878,314.2784,0.000000,...,0.306633,2,1,247.4366,1.025214,186.3625,0.000000,1,1,1
3,2015-12-22 16:30:03,2.428658,512.0915,2,2,1,178.8644,8.590618,315.7652,2.449108,...,0.306633,2,1,250.6729,0.945119,188.9259,0.000000,1,1,1
4,2015-12-22 16:30:04,2.545239,505.3008,2,2,1,264.4194,8.348372,331.4535,2.429501,...,0.305480,2,1,246.5714,1.105309,185.8177,0.000064,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545097,2015-12-28 23:54:57,0.000000,813.0031,1,1,1,266.6624,8.446745,319.3284,0.000000,...,0.308491,2,1,251.1856,1.057252,190.0154,0.000064,1,1,1
545098,2015-12-28 23:54:58,0.000000,681.7811,1,2,1,229.7167,8.466612,332.6583,2.467305,...,0.308426,2,1,250.6248,0.688816,189.0220,0.000000,1,1,1
545099,2015-12-28 23:54:59,2.515453,518.5289,2,2,1,261.9200,8.378813,329.7616,2.427450,...,0.306761,2,1,249.8398,1.185403,188.7337,0.000000,1,1,1
545100,2015-12-28 23:55:00,2.523460,518.9214,2,2,1,177.4225,8.611446,303.7682,2.445904,...,0.306633,2,1,250.9773,0.816967,188.8458,0.000000,1,1,1


In [292]:
y_test = df_test_baseline[270602:]['IsAttack']
X_test = df_test_baseline[270602:]

In [293]:
X_test.drop(columns='IsAttack', inplace=True)
X_test = X_test.reset_index(drop=True)
X_test

,Timestamp,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,...,FIT504,P501,P502,PIT501,PIT502,PIT503,FIT601,P601,P602,P603
0,2015-12-28 23:55:02,0.000000,627.4946,1,2,1,258.1389,8.379133,333.7862,2.455900,...,0.306761,2,1,247.7890,0.961138,186.8591,0.000064,1,1,1
1,2015-12-28 23:55:03,2.481504,497.2147,2,2,1,261.0228,8.389067,331.0433,2.440778,...,0.306889,2,1,251.0574,1.730048,189.8391,0.000128,1,1,1
2,2015-12-28 23:55:04,2.552926,563.9444,2,1,1,218.4055,8.569469,312.6891,0.000000,...,0.308362,2,1,250.5768,1.457726,189.0861,1.737161,1,2,1
3,2015-12-28 23:55:05,0.000000,630.9489,1,2,1,198.3145,8.538708,323.8913,2.458335,...,0.304839,2,1,245.6902,1.345593,184.6321,0.000064,1,1,1
4,2015-12-28 23:55:06,0.330846,808.0180,1,1,1,263.4581,8.415342,320.2256,0.000000,...,0.306825,2,1,250.0000,1.169384,189.2143,0.000064,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399791,2016-01-02 14:59:34,2.668866,509.8933,2,2,1,271.0844,8.350295,329.9667,2.448595,...,0.306889,2,1,253.3165,1.105309,192.0822,0.000064,1,1,1
399792,2016-01-02 14:59:35,2.571182,750.1987,2,1,1,183.3825,8.672008,302.7429,0.000000,...,0.307594,2,1,250.3044,1.009195,188.4132,0.000000,1,1,1
399793,2016-01-02 14:59:36,2.564135,501.3755,2,2,1,264.4514,8.358306,331.6073,2.423606,...,0.306761,2,1,247.8531,1.089290,187.1155,0.000000,1,1,1
399794,2016-01-02 14:59:37,2.555168,671.2614,2,1,1,176.1087,8.663676,300.0513,0.000000,...,0.306633,2,1,252.2751,0.800948,190.0955,0.000000,1,1,1


In [294]:
X_train.drop(columns='Timestamp', inplace=True)
X_test.drop(columns='Timestamp', inplace=True)

In [295]:
from pyod.models.ecod import ECOD
from pyod.utils.data import generate_data
from pyod.utils.data import evaluate_print

ecod_model = ECOD()

ecod_model.fit(X_train)

y_train_preds = ecod_model.labels_

y_train_Scores = ecod_model.decision_scores_

y_test_preds = ecod_model.predict(X_test)

y_test_Scores = ecod_model.decision_function(X_test)

print("\nOn Training Data:")
evaluate_print('ECOD', y_train, y_train_Scores)
print("\nOn Test Data:")
evaluate_print('ECOD', y_test, y_test_Scores)


On Training Data:
ECOD ROC:0.8815, precision @ rank n:0.6323

On Test Data:
ECOD ROC:0.8791, precision @ rank n:0.66


In [126]:
from pyod.models.copod import COPOD
from pyod.utils.data import generate_data
from pyod.utils.data import evaluate_print

copod_model = COPOD()

copod_model.fit(X_train)

y_train_preds = copod_model.labels_

y_train_Scores = copod_model.decision_scores_

y_test_preds = copod_model.predict(X_test)

y_test_Scores = copod_model.decision_function(X_test)

print("\nOn Training Data:")
evaluate_print('COPOD', y_train, y_train_Scores)
print("\nOn Test Data:")
evaluate_print('COPOD', y_test, y_test_Scores)


On Training Data:
COPOD ROC:0.8834, precision @ rank n:0.6558

On Test Data:
COPOD ROC:0.8803, precision @ rank n:0.6667


In [131]:
from pyod.models.sampling import Sampling
from pyod.utils.data import generate_data
from pyod.utils.data import evaluate_print

sampling_model = Sampling()

sampling_model.fit(X_train)

y_train_preds = sampling_model.labels_

y_train_Scores = sampling_model.decision_scores_

y_test_preds = sampling_model.predict(X_test)

y_test_Scores = sampling_model.decision_function(X_test)

print("\nOn Training Data:")
evaluate_print('Sampling', y_train, y_train_Scores)
print("\nOn Test Data:")
evaluate_print('Sampling', y_test, y_test_Scores)


On Training Data:
Sampling ROC:0.9022, precision @ rank n:0.6483

On Test Data:
Sampling ROC:0.9079, precision @ rank n:0.6939


In [132]:
from pyod.models.inne import INNE
from pyod.utils.data import generate_data
from pyod.utils.data import evaluate_print

inne_model = INNE()

inne_model.fit(X_train)

y_train_preds = inne_model.labels_

y_train_Scores = inne_model.decision_scores_

y_test_preds = inne_model.predict(X_test)

y_test_Scores = inne_model.decision_function(X_test)

print("\nOn Training Data:")
evaluate_print('INNE', y_train, y_train_Scores)
print("\nOn Test Data:")
evaluate_print('INNE', y_test, y_test_Scores)


On Training Data:
INNE ROC:0.8859, precision @ rank n:0.4192

On Test Data:
INNE ROC:0.9026, precision @ rank n:0.6826


In [ ]:
from_pandas_dynamic()